<a href="https://colab.research.google.com/github/srJboca/segmentacion/blob/main/ES/3.%20Segmentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Segmentación de Clientes de Gas

## Introducción

Este notebook se enfoca en la segmentación de clientes utilizando el DataFrame `df_analisis.parquet`, que fue preparado en el notebook de exploración de datos. La segmentación de clientes nos permite agrupar clientes con características similares, lo cual es útil para estrategias de marketing personalizadas, gestión de servicios y optimización de operaciones.

**Objetivo:** Identificar distintos segmentos de clientes basados en su comportamiento de consumo y pago.

**Técnicas Utilizadas:**
1.  **Agregación de Datos:** Transformar datos a nivel de factura en características a nivel de cliente.
2.  **Análisis de Componentes Principales (PCA):** Reducir la dimensionalidad de los datos para facilitar la visualización y el clustering.
3.  **K-Means Clustering:** Agrupar los clientes en K segmentos distintos.
4.  **Perfilado de Segmentos:** Analizar las características de cada segmento.

## 1. Configuración del Entorno y Carga de Datos

### 1.1 Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### 1.2 Descarga y Carga del DataFrame Preprocesado

Utilizaremos el archivo `df_analisis.parquet`.

In [ ]:
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/df_analisis.parquet
df_analisis = pd.read_parquet('df_analisis.parquet')

## 2. Revisión Rápida de Datos
Recordemos la estructura del DataFrame `df_analisis`.

In [ ]:
print("--- Primeras 5 filas de df_analisis ---")
print(df_analisis.head())
print("\n--- Información de df_analisis ---")
df_analisis.info()

## 3. Agregación de Características a Nivel de Cliente

Para segmentar clientes, necesitamos características que describan el comportamiento de cada cliente a lo largo del tiempo. Agruparemos los datos por `Numero de contrato` y calcularemos métricas agregadas (promedios) de su consumo, costos y comportamiento de pago.

El DataFrame `df_analisis` original (celda 4 del notebook proporcionado) contiene datos a nivel de factura. El notebook original también crea un `df_prediccion` (celda 6) que selecciona ciertas columnas, pero la agregación (celda 8) se realiza directamente sobre `df_analisis`. Seguiremos esta última lógica.

In [ ]:
df_grouped = df_analisis.groupby('Numero de contrato').agg(
    Consumo_Promedio=('Consumo (m3)', 'mean'),
    Precio_Consumo_Promedio=('Precio por Consumo', 'mean'),
    Dias_Emision_PagoOportuno_Promedio=('Dias_Emision_PagoOportuno', 'mean'),
    Dias_Lectura_Emision_Promedio=('Dias_Lectura_Emision', 'mean'),
    Dias_PagoOportuno_PagoReal_Promedio=('Dias_PagoOportuno_PagoReal', 'mean'),
    Mora_Promedio=('Mora', 'mean') # Tasa de mora
).reset_index()

print("--- df_grouped (datos agregados por cliente) ---")
print(df_grouped.head())

### 3.1 Incorporar 'Estrato Socioeconómico'

El 'Estrato socioeconomico' es una característica importante del cliente. Lo uniremos al DataFrame agrupado. Necesitamos obtener el estrato de `df_analisis` (o podríamos haberlo obtenido de `df_clientes` en la fase de exploración). Como `df_analisis` ya tiene 'Estrato', lo usaremos, asegurándonos de tomar un único valor por cliente.

In [ ]:
# Obtener el estrato para cada contrato (tomando el primero, asumiendo que no cambia)
df_estratos = df_analisis.drop_duplicates(subset=['Numero de contrato'])[['Numero de contrato', 'Estrato']].copy()
# Renombrar 'Estrato' a 'Estrato socioeconomico' para claridad si es necesario
df_estratos.rename(columns={'Estrato': 'Estrato socioeconomico'}, inplace=True)

df_segmentacion = pd.merge(df_grouped, df_estratos, on='Numero de contrato', how='left')

print("--- df_segmentacion con Estrato ---")
print(df_segmentacion.head())
print("\n--- Información de df_segmentacion ---")
df_segmentacion.info()

## 4. Preprocesamiento para PCA y Clustering

### 4.1 Selección de Características Numéricas y Manejo de NaNs
Seleccionaremos las características numéricas para PCA y clustering. La columna 'Estrato socioeconomico' es categórica y la trataremos después o la usaremos para perfilar los segmentos. Por ahora, la convertiremos a numérica si es ordinal.
También eliminaremos filas con valores NaN en las características seleccionadas.

In [ ]:
# Convertir 'Estrato socioeconomico' a numérico (ordinal)
if df_segmentacion['Estrato socioeconomico'].dtype == 'object' or isinstance(df_segmentacion['Estrato socioeconomico'].dtype, pd.CategoricalDtype):
    df_segmentacion['Estrato_Num'] = df_segmentacion['Estrato socioeconomico'].str.replace('Estrato ', '', regex=False).astype(int)
else:
    df_segmentacion['Estrato_Num'] = df_segmentacion['Estrato socioeconomico'].astype(int)

features_for_pca = [
    'Consumo_Promedio',
    'Precio_Consumo_Promedio',
    'Dias_Emision_PagoOportuno_Promedio',
    'Dias_Lectura_Emision_Promedio',
    'Dias_PagoOportuno_PagoReal_Promedio',
    'Mora_Promedio',
    'Estrato_Num' # Incluimos el estrato numérico
]
X = df_segmentacion[features_for_pca].copy()

print(f"Shape antes de dropna: {X.shape}")
X.dropna(inplace=True) # Eliminar filas con NaNs en estas características
print(f"Shape después de dropna: {X.shape}")

print("\nValores faltantes después de dropna:")
print(X.isnull().sum())

### 4.2 Escalado de Características
PCA es sensible a la escala de las características. Por lo tanto, estandarizaremos los datos.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("--- Datos escalados (primeras 5 filas) ---")
print(pd.DataFrame(X_scaled, columns=X.columns).head())

## 5. Análisis de Componentes Principales (PCA)

Reduciremos la dimensionalidad a 2 componentes principales para visualización.

In [ ]:
pca = PCA(n_components=2) # Reducir a 2 componentes
X_pca = pca.fit_transform(X_scaled)

df_pca = pd.DataFrame(data=X_pca, columns=['principal_component_1', 'principal_component_2'])

print("--- Componentes Principales (primeras 5 filas) ---")
print(df_pca.head())

print(f"\nVarianza explicada por cada componente: {pca.explained_variance_ratio_}")
print(f"Varianza explicada total (2 componentes): {pca.explained_variance_ratio_.sum():.2f}")

La varianza explicada total nos dice qué porcentaje de la información original se retiene en los 2 componentes principales.

## 6. K-Means Clustering

### 6.1 Método del Codo para K Óptimo
Usaremos el método del codo para ayudar a determinar un número apropiado de clusters (K). Buscamos el punto donde añadir más clusters no mejora significativamente la suma de los errores cuadráticos (SSE) o inercia.

In [ ]:
sse = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto') # n_init='auto' es el valor por defecto en versiones recientes
    kmeans.fit(df_pca) # Usamos los datos transformados por PCA
    sse.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, sse, marker='o')
plt.title('Método del Codo para Encontrar K Óptimo')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('SSE (Inercia)')
plt.xticks(k_range)
plt.grid(True)
plt.show()

Observa el gráfico anterior. El "codo" es el punto donde la tasa de disminución de SSE se aplana. Este punto sugiere un valor K óptimo. Para este tutorial, basándonos en el notebook original, usaremos K=4. (En un caso real, seleccionarías K basado en la visualización del codo).

### 6.2 Aplicación de K-Means y Visualización
Aplicamos K-Means con el K óptimo (K=4) y visualizamos los clusters en el espacio de PCA.

In [ ]:
optimal_k = 4 # @param {type:"integer"} # Elegido del método del codo o análisis de negocio

kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=42, n_init='auto')
df_pca['segmento'] = kmeans_optimal.fit_predict(df_pca)

plt.figure(figsize=(12, 8))
sns.scatterplot(x='principal_component_1', y='principal_component_2', hue='segmento', data=df_pca, palette='viridis', s=100, alpha=0.7)
plt.title(f'Clusters de Clientes (k={optimal_k}) en Espacio PCA')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Segmento de Cliente')
plt.grid(True)
plt.show()

print("--- df_pca con Segmentos ---")
print(df_pca.head())

## 7. Perfilado de Segmentos

Ahora que tenemos los segmentos, necesitamos entender qué caracteriza a cada uno. Uniremos las etiquetas de segmento de nuevo al DataFrame `df_segmentacion` (que contiene las características agregadas y el estrato) y analizaremos las medias de las características para cada segmento.

In [ ]:
# X fue X.dropna(), así que df_pca debería tener la misma longitud y orden
# Creamos df_segmentacion_cleaned para asegurar la correspondencia de índices
df_segmentacion_cleaned = df_segmentacion.loc[X.index].copy()
df_segmentacion_cleaned['segmento'] = df_pca['segmento'].values

print("--- df_segmentacion_cleaned con Segmentos ---")
print(df_segmentacion_cleaned.head())

### 7.1 Características Promedio por Segmento

In [ ]:
segment_profiles = df_segmentacion_cleaned.groupby('segmento')[features_for_pca].mean()
print("--- Perfiles de Segmentos (Características Promedio) ---")
print(segment_profiles)

# Visualización de los perfiles
segment_profiles.T.plot(kind='bar', figsize=(15, 7))
plt.title('Características Promedio por Segmento de Cliente')
plt.ylabel('Valor Promedio Escalado (o original si se usa X)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Segmento')
plt.tight_layout()
plt.show()

### 7.2 Distribución de Estrato por Segmento

In [ ]:
estrato_segment_distribution = pd.crosstab(df_segmentacion_cleaned['segmento'], df_segmentacion_cleaned['Estrato socioeconomico'], normalize='index') * 100
print("\n--- Distribución Porcentual de Estrato por Segmento ---")
print(estrato_segment_distribution)

estrato_segment_distribution.plot(kind='bar', stacked=True, figsize=(12, 7), colormap='viridis')
plt.title('Distribución de Estrato Socioeconómico por Segmento')
plt.xlabel('Segmento de Cliente')
plt.ylabel('Porcentaje de Clientes (%)')
plt.xticks(rotation=0)
plt.legend(title='Estrato')
plt.tight_layout()
plt.show()

**Interpretación de Perfiles:**
* **Características Promedio:** Observa las barras para cada segmento. ¿Qué características tienen valores altos o bajos en un segmento particular? Por ejemplo, un segmento podría tener un 'Consumo_Promedio' alto y una 'Mora_Promedio' baja, indicando clientes de alto valor y buen comportamiento de pago.
* **Distribución de Estrato:** ¿Hay algún estrato predominante en cada segmento? Esto puede ayudar a entender la composición socioeconómica de los grupos.

**Nombrando los Segmentos (Ejemplo):**
Basado en el análisis, podrías asignar nombres descriptivos a cada segmento, como:
* **Segmento 0:** Clientes de Bajo Consumo, Pago Puntual
* **Segmento 1:** Clientes de Alto Consumo, Morosidad Moderada, Estrato Alto
* Etc.

## 8. Conclusiones y Próximos Pasos

En este notebook, hemos:
1.  Agregado los datos a nivel de cliente.
2.  Utilizado PCA para reducir la dimensionalidad y facilitar la visualización.
3.  Aplicado K-Means para agrupar a los clientes en segmentos (K=4 en este ejemplo).
4.  Iniciado el perfilado de los segmentos analizando sus características promedio y la distribución de estratos.

**Próximos Pasos Potenciales:**
* **Validación de Segmentos:** Utilizar métricas de clustering (como el Coeficiente de Silueta) para evaluar la calidad de los segmentos. También, validar la estabilidad de los segmentos con diferentes inicializaciones de K-Means o subconjuntos de datos.
* **Perfilado Detallado:** Analizar más variables para cada segmento (e.g., antigüedad del cliente, tipo de contrato, si se incluyeran).
* **Acciones Estratégicas:** Definir estrategias específicas para cada segmento (e.g., campañas de fidelización para clientes de alto valor, programas de gestión de mora para segmentos con alta tasa de impago, ofertas personalizadas según el consumo).
* **Monitoreo:** Observar cómo evolucionan los segmentos con el tiempo y si los clientes cambian de un segmento a otro.
* **Probar otros algoritmos:** Explorar otros algoritmos de clustering (e.g., DBSCAN, Aglomerativo) si K-Means no produce resultados satisfactorios o si se sospecha de estructuras de clusters no esféricas.

La segmentación de clientes es una herramienta poderosa para entender mejor la base de clientes y tomar decisiones de negocio más informadas.